In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset 
from PIL import Image
import numpy as np
from PIL import ImageFile
import csv
import  tqdm
 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn.functional as F
from IPython.display import display

In [ ]:
#Here I will be creating a dataloader which has the images that we want to reconstruct. 
#We can also pass singluar images to the networks after converting then to tensors and normalizing them.
#We could have also made a testloader which is the more common and logical way to do it.
#But if you have a bunch of photos of some friends or something, you can keep them in a folder and make a dataloader as I have done here


In [ ]:
class data_set(Dataset):
    def __init__(self , csv_file  , root_dir , transform = None ):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
        
        
    def __len__(self):
        return len(self.annotations) 
    
    
    def __getitem__(self , index):
        try:
            img_path = os.path.join(self.root_dir , self.annotations.iloc[index , 0])
            image = Image.open(img_path )

            if self.transform:
                image = self.transform(image)

      
        except:
            return None

        return image

batch_size = 1
def collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))
    return torch.utils.data.dataloader.default_collate(batch)
        

In [ ]:

ImageFile.LOAD_TRUNCATED_IMAGES = True

my_transforms = transforms.Compose([
    #transforms.ToPILImage() , 
    transforms.Resize((64,64)) , 
    transforms.ToTensor() , 
    transforms.Normalize((0.0 , 0.0 , 0.0), (1.0 , 1.0 , 1.0)),
])

data=[]
i = 0
with open('test_image_annotation.csv', 'w', newline='') as writeFile:
    writer = csv.writer(writeFile)
    
    for filename in os.listdir(r"C:\Users\Abhrant\Desktop\abhrant\work\DEEP_LEARNING\FaceDataset"):
        
            data.append(filename)
            writer.writerow(data)
            data=[]
            i += 1
            if i >= 100:
                break
writeFile.close()

colored_dataset = data_set(csv_file = "test_image_annotation.csv" , root_dir = r"C:\Users\Abhrant\Desktop\abhrant\work\DEEP_LEARNING\FaceDataset" , 
                          transform = my_transforms)



test_image_loader = DataLoader(dataset = colored_dataset , collate_fn = collate_fn ,  batch_size = batch_size , shuffle = False)

In [ ]:
state_dict = torch.load(r"C:\Users\Abhrant\Desktop\abhrant\work\DEEP_LEARNING\vae_weight.pt" , 
             map_location = 'cuda:0')
vae.load_state_dict(state_dict)
vae.eval()
vae.cuda()

for i in (test_image_loader):
    
    input_img = i
    output , mu , logvar = vae.forward(i.to("cuda"))
    
    recon_img = output.to("cpu")
    recon_img = recon_img.view(3 , 64 , 64)
    to_pil = torchvision.transforms.ToPILImage()
    recon_img = to_pil(recon_img)
    
    input_img = input_img.to("cpu")
    input_img = input_img.view(3 , 64 , 64)
    input_img = to_pil(input_img)
    
    display(input_img , recon_img)
    
    
    